In [1]:

import  numpy as np
from sklearn import datasets
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
from sklearn.svm import SVC
import xgboost as xgb 
from sklearn import metrics as m
from thundersvm import SVC as svmgpu
import calculateWeightUsingGa2 as aresult
import pandas as pd
import itertools
import random
from sklearn.utils import shuffle
from sklearn.preprocessing import LabelEncoder

def swapcolumns(trainval, testval, coldindexval):
    trainval[trainval != coldindexval] = 5
    testval[testval != coldindexval] = 5

    trainval[trainval == coldindexval] = 0
    trainval[trainval == 5] = 1

    testval[testval == coldindexval] = 0
    testval[testval == 5] = 1

    return trainval, testval


randomseed=42
np.random.seed(randomseed)

In [2]:

xtest=np.array(pd.read_csv('../dataset/xtest.txt'))
xtrain=np.array(pd.read_csv('../dataset/xtrain.txt'))
ytest_original=np.array(pd.read_csv('../dataset/ytest.txt')).ravel()
ytrain_original =np.array(pd.read_csv('../dataset/ytrain.txt')).ravel()

# data=datasets.load_wine()
# x=data.data
# y=data.target

# data = pd.read_csv("../dataset/seeds_dataset.txt", sep="\t", header=None)
# data = shuffle(data)
# le = LabelEncoder()
# data.iloc[:, -1] = le.fit_transform(data.iloc[:, -1])
# x = data.iloc[:, :-1]
# y = data.iloc[:, -1]
# print(np.unique(y))

# data = pd.read_csv("../dataset/ionosphere.data",  header=None)
# data = shuffle(data)
# le = LabelEncoder()
# data.iloc[:, -1] = le.fit_transform(data.iloc[:, -1])
# x = data.iloc[:, :-1]
# y = data.iloc[:, -1]
# print(np.unique(y))




# xtrain,xtest,ytrain_original,ytest_original=train_test_split(x,y,random_state=randomseed,test_size=0.3) 

ytrain=ytrain_original.copy()
ytest=ytest_original.copy() 

# ytrain, ytest = swapcolumns(ytrain, ytest, 2)

In [3]:


clf=[]
acc=[]
finalacc=[]
ypredproba_all=[]
ypredconfprob_all=[]


#================================================= 

# Class 0
# ===========================
ytrain = ytrain_original.copy()
ytest = ytest_original.copy()
ytrain,ytest= swapcolumns(ytrain,ytest,1)
#=================================================

rf=RandomForestClassifier(random_state=randomseed, n_estimators=10)
rf.fit(xtrain,ytrain)
print('original score',m.f1_score(ytest,rf.predict(xtest),average='weighted'))


xgbc=xgb.XGBClassifier(random_state=randomseed,n_estimators=100)
xgbc.fit(xtrain,ytrain)
xgbpred=xgbc.predict(xtest)
print('original score 2',m.f1_score(ytest,xgbpred,average='weighted'))


#================================================= 

rf=RandomForestClassifier(random_state=randomseed, n_estimators=10)
rf.fit(xtrain,ytrain)
rfpred=rf.predict(xtest)
print(m.f1_score(ytest,rfpred,average='weighted'))

clf.append(rf)
acc.append(m.f1_score(ytest,rfpred,average='weighted'))
ypredproba_all.append(rf.predict_proba(xtest)[:,0])

confmat = m.confusion_matrix(ytest, rfpred)
confsumh = np.sum(confmat, axis=0)
propconfmat = confmat.copy()
for i in range(propconfmat.shape[0]):
    propconfmat[:, i] = 100 * propconfmat[:, i] / confsumh[i]
ypredconfprob_all.append(propconfmat.ravel() / 100)


#=================================================
svc=svmgpu(random_state=randomseed,probability=True,C=100,gamma=0.0001)
svc.fit(xtrain,ytrain)

svcpred=svc.predict(xtest)
print(m.f1_score(ytest,svcpred,average='weighted'))

clf.append(svc)
acc.append(m.f1_score(ytest,svcpred,average='weighted'))
ypredproba_all.append(svc.predict_proba(xtest)[:,1])

confmat = m.confusion_matrix(ytest, rfpred)
confsumh = np.sum(confmat, axis=0)
propconfmat = confmat.copy()
for i in range(propconfmat.shape[0]):
    propconfmat[:, i] = 100 * propconfmat[:, i] / confsumh[i]
ypredconfprob_all.append(propconfmat.ravel() / 100)

#=================================================
xgbc=xgb.XGBClassifier(random_state=randomseed,n_estimators=100)
xgbc.fit(xtrain,ytrain)

xgbpred=xgbc.predict(xtest)
print(m.f1_score(ytest,xgbpred,average='weighted'))


clf.append(xgbc)
acc.append(m.f1_score(ytest,xgbpred,average='weighted'))
ypredproba_all.append(xgbc.predict_proba(xtest)[:,0])

confmat = m.confusion_matrix(ytest, rfpred)
confsumh = np.sum(confmat, axis=0)
propconfmat = confmat.copy()
for i in range(propconfmat.shape[0]):
    propconfmat[:, i] = 100 * propconfmat[:, i] / confsumh[i]
ypredconfprob_all.append(propconfmat.ravel() / 100)

#=================================================
#=================================================
# generate combinations of features 12,6
comb = list(itertools.combinations(np.arange(0, 12, 1), 7))

# generate 50 random numbers
randnums = []
for i in range(50):
    randnums.append(random.randrange(0, len(comb)))

print(randnums)

comb = np.array(comb)[randnums, :]


for i in range(len(comb)):
    print(i, " ==================== ", comb[i])

    rf = RandomForestClassifier(random_state=randomseed, n_estimators=50)
    rf.fit(xtrain[:, comb[i]], ytrain)
    rfpred = rf.predict(xtest[:, comb[i]])
    print(m.f1_score(ytest, rfpred,average='weighted'))

    clf.append(rf)
    acc.append(m.f1_score(ytest, rfpred,average='weighted'))
    ypredproba_all.append(rf.predict_proba(xtest[:, comb[i]])[:,0])

    confmat = m.confusion_matrix(ytest, rfpred)
    confsumh = np.sum(confmat, axis=0)
    propconfmat = confmat.copy()
    for i in range(propconfmat.shape[0]):
        propconfmat[:, i] = 100 * propconfmat[:, i] / confsumh[i]
    ypredconfprob_all.append(propconfmat.ravel() / 100)

    xgbmodel = xgb.XGBClassifier(random_state=randomseed, n_estimators=50)
    xgbmodel.fit(xtrain, ytrain)
    xgbmodelpred = xgbmodel.predict(xtest)
    print(m.f1_score(ytest, xgbmodelpred,average='weighted'))

    clf.append(xgbmodel)
    acc.append(m.f1_score(ytest, xgbmodelpred,average='weighted'))
    ypredproba_all.append(xgbmodel.predict_proba(xtest)[:,0])

    confmat = m.confusion_matrix(ytest, xgbmodelpred)
    confsumh = np.sum(confmat, axis=0)
    propconfmat = confmat.copy()
    for i in range(propconfmat.shape[0]):
        propconfmat[:, i] = 100 * propconfmat[:, i] / confsumh[i]
    ypredconfprob_all.append(propconfmat.ravel() / 100)

# #=================================================


original score 0.6765880295918836
original score 2 0.7315606204495092
0.6765880295918836
0.5333333333333333
0.7315606204495092
[651, 673, 246, 521, 9, 19, 43, 24, 633, 245, 772, 106, 509, 461, 288, 224, 404, 316, 536, 106, 695, 507, 589, 586, 69, 259, 606, 773, 260, 261, 517, 218, 373, 636, 608, 113, 174, 759, 788, 179, 87, 633, 510, 738, 255, 265, 453, 299, 777, 399]
0  ====================  [ 1  4  5  6  8  9 11]
0.6719547585385817
0.738221824835998
1  ====================  [2 3 4 5 6 7 9]
0.5554454483629652
0.738221824835998
2  ====================  [ 0  1  6  7  8  9 10]
0.6909657774380997
0.738221824835998
3  ====================  [ 1  2  3  6  7  9 11]
0.6671622795018497
0.738221824835998
4  ====================  [ 0  1  2  3  4  6 10]
0.7195062755703487
0.738221824835998
5  ====================  [ 0  1  2  3  4  9 11]
0.7133949365186689
0.738221824835998
6  ====================  [ 0  1  2  3  6  9 10]
0.7365201995815226
0.738221824835998
7  ====================  [ 0  1  2  3  5 

In [4]:
# np.savetxt("foo.csv", np.array(ypredproba_all), delimiter=",")

In [5]:
# pd_clf=pd.DataFrame(list(clf))
pd_acc=pd.DataFrame(acc) 
pd_ypredproba_all=pd.DataFrame(np.array(ypredproba_all))
pd_ypredconfprob_all=pd.DataFrame(ypredconfprob_all)

In [6]:
classname='class1.txt'
pd_acc.to_csv('pd_acc_'+classname,sep=',',index=False) 
pd_ypredproba_all.to_csv('pd_ypredproba_all_'+classname,sep=',',index=False)
pd_ypredconfprob_all.to_csv('pd_ypredconfprob_all_'+classname,sep=',',index=False)

In [7]:
pd.DataFrame(ytest_original).to_csv('ytest_original.txt',sep=',',index=False)
pd.DataFrame(ytest).to_csv('ytest_class1.txt',sep=',',index=False)